<a href="https://colab.research.google.com/github/pof1423/Socar-bootcamp/blob/main/first_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
tada_eta = pd.read_excel('파일경로')
print(tada_eta.columns)
tada_eta


Index(['ATA', 'id', 'api_eta', 'created_at_kst', 'month', 'pickup_lng',
       'pickup_lat', 'pickup_gu', 'driver_id', 'driver_lng', 'driver_lat',
       'hour'],
      dtype='object')


,ATA,id,api_eta,created_at_kst,month,pickup_lng,pickup_lat,pickup_gu,driver_id,driver_lng,driver_lat,hour
0,10.60,R4L9FL6NVEL4D0,5.65,2019-12-01T00:55:09,12,127.091399,37.624545,노원구,DST52944,127.069978,37.616472,0
1,9.20,R3A7JBY7CS6TPD,8.60,2019-12-01T00:09:39,12,127.023351,37.625564,강북구,DPC72995,127.026479,37.604711,0
2,10.85,R2BIUY1TDK14XE,10.53,2019-12-15T00:18:42,12,126.881154,37.481641,금천구,DPP94923,126.906225,37.452167,0
3,5.52,R31PU5GNHSGU6C,1.92,2019-07-31T00:24:11,7,126.988375,37.567897,중구,DCT73585,126.991170,37.566416,0
4,8.60,R41WR6N1VT42DJ,7.10,2019-07-05T00:40:34,7,126.988684,37.566716,중구,DPJ86237,127.006860,37.571164,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13911,8.22,R2GDE24ME5JVNW,5.55,2019-12-21T23:20:13,12,126.909651,37.521901,영등포구,DST69352,126.916539,37.527159,23
13912,1.37,RAS6GH62M4SZH,0.98,2019-07-16T23:41:45,7,126.901157,37.534355,영등포구,DBL73640,126.904703,37.532614,23
13913,4.22,RRJJ2J7GZIZ6Y,4.15,2019-07-23T23:25:47,7,126.925215,37.524155,영등포구,DRD21655,126.921785,37.521985,23
13914,4.85,R4NN4WYS6FTWHD,5.38,2019-07-01T23:27:23,7,126.928785,37.528182,영등포구,DMJ83759,126.955591,37.532395,23


In [ ]:
is_na = tada_eta.isna().sum(0)
is_na

ATA               0
id                0
api_eta           0
created_at_kst    0
month             0
pickup_lng        0
pickup_lat        0
pickup_gu         0
driver_id         0
driver_lng        0
driver_lat        0
hour              0
dtype: int64

In [ ]:
for col in tada_eta.columns:
  print(f"Data type for column {col} is {tada_eta[col].dtype}.")

Data type for column ATA is float64.
Data type for column id is object.
Data type for column api_eta is float64.
Data type for column created_at_kst is object.
Data type for column month is int64.
Data type for column pickup_lng is float64.
Data type for column pickup_lat is float64.
Data type for column pickup_gu is object.
Data type for column driver_id is object.
Data type for column driver_lng is float64.
Data type for column driver_lat is float64.
Data type for column hour is int64.


In [ ]:
#distance를 위경도로 기반해서 km로 변환
x_1 = ((np.cos(tada_eta['pickup_lat']) * 6400 * 2 * np.pi) / 360) * np.absolute(tada_eta['pickup_lng'] - tada_eta['driver_lng'])
y_1 = 111 * np.absolute(tada_eta['pickup_lat'] - tada_eta['driver_lat'])
tada_eta['distance'] = np.sqrt(x_1**2+y_1**2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
input_labels = ['강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구','성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구']
encoder = LabelEncoder()

encoder.fit(input_labels)
tada_eta['pickup_gu'] = encoder.transform(tada_eta['pickup_gu'])

In [ ]:
tada_eta = tada_eta.drop(columns=['id','created_at_kst','pickup_lng','pickup_lat','driver_id','driver_lng','driver_lat'])

In [ ]:
tada_eta

,ATA,api_eta,month,pickup_gu,hour,distance
0,10.60,5.65,12,8,0,2.548743
1,9.20,8.60,12,2,0,2.340836
2,10.85,10.53,12,7,0,4.263972
3,5.52,1.92,7,23,0,0.350467
4,8.60,7.10,7,23,0,2.072178
...,...,...,...,...,...,...
13911,8.22,5.55,12,19,23,0.956072
13912,1.37,0.98,7,19,23,0.435926
13913,4.22,4.15,7,19,23,0.447609
13914,4.85,5.38,7,19,23,2.987416


#정규화- 했더니 약간 값이 이상하게 나옴...
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(tada_eta[['ATA','api_eta', 'hour', 'distance']])
tada_scaled= scaler.transform(tada_eta[['ATA','api_eta', 'hour', 'distance']])

tada_eta_cat = tada_eta[['month', 'pickup_gu']]

df_tada = pd.DataFrame(data=tada_scaled, columns=['ATA', 'api_eta', 'hour', 'distance'])


df_tada = pd.concat([df_tada, tada_eta_cat], axis=1)
df_tada

#정규화 했을시 테스트셋, 트레인 셋 구분
df_tada = df_tada.sample(frac=1, random_state=0).reset_index(drop=True)
train = df_tada[:12000]
test = df_tada[12000:]
x_train = np.asarray(train.drop('ATA',1))
y_train = np.asarray(train['ATA'])
x_test = np.asarray(test.drop('ATA',1))
y_test = np.asarray(test['ATA'])


In [ ]:
#히스트그래디언트부스팅 모델 중 regressor 선택
from sklearn import ensemble
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
hgbm_reg = sklearn.ensemble.HistGradientBoostingRegressor()

In [ ]:
tada_eta = tada_eta.sample(frac=1, random_state=0).reset_index(drop=True)
train = tada_eta[:12000]
test = tada_eta[12000:]
x_train = np.asarray(train.drop('ATA',1))
y_train = np.asarray(train['ATA'])
x_test = np.asarray(test.drop('ATA',1))
y_test = np.asarray(test['ATA'])

In [ ]:
#랜덤 서치로 트리 갯수, 트리 깊이, 잎의 최소 데이터 수, 학습률 등을 설정
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [10,30,50,70,90,100,1000,10000]
max_depth = [np.arange(1,20,3)]
min_samples_leaf = [np.arange(1,20,3)]
learning_rate = [0.001,0.005,0.01,0.05,0.1,0.5,0.7]
random_grid = {'max_iter': [10,30,50,70,90,100,1000,10000],
               'max_depth': np.arange(1,20,3),
               'min_samples_leaf' : np.arange(1,20,3),
               'learning_rate': [0.001,0.005,0.01,0.05,0.1,0.5,0.7]}

In [ ]:
# 랜덤서치 모델 튜딩 후 fitting
rnd_search = RandomizedSearchCV(hgbm_reg, param_distributions=random_grid, n_iter=50, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
rnd_search.fit(x_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=HistGradientBoostingRegressor(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.001, 0.005, 0.01,
                                                          0.05, 0.1, 0.5, 0.7],
                                        'max_depth': array([ 1,  4,  7, 10, 13, 16, 19]),
                                        'max_iter': [10, 30, 50, 70, 90, 100,
                                                     1000, 10000],
                                        'min_samples_leaf': array([ 1,  4,  7, 10, 13, 16, 19])},
                   scoring='neg_mean_squared_error', verbose=1)

In [ ]:
print(rnd_search.best_params_) # 가장 성능 좋은 파라미터 찾기

{'min_samples_leaf': 7, 'max_iter': 1000, 'max_depth': 4, 'learning_rate': 0.005}


In [ ]:
params = {'max_iter': 1000,
          'max_depth': 4,
           'min_samples_leaf': 7,
          'learning_rate': 0.005}

In [ ]:
hgbm_reg_best = sklearn.ensemble.HistGradientBoostingRegressor(**rnd_search.best_params_)

In [ ]:
hgbm_reg_best.fit(x_train, y_train)
hgbm_reg_best.fit(x_test, y_test)
mse = mean_squared_error(y_test, hgbm_reg_best.predict(x_test)) # 테스트 셋에 모델 피팅
print(mse)

6.707382051874374


In [ ]:
print(hgbm_reg_best.score(x_train, y_train))
print(hgbm_reg_best.score(x_test, y_test))

0.4972161941796468
0.6012539550346174


In [ ]:
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
print("The initial error of API ETA on test set: {:.4f}".format(mean_squared_error(y_test, x_test[:,0]) ))

The mean squared error (MSE) on test set: 6.7074
The initial error of API ETA on test set: 13.7135
